In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_colwidth', None)
train= pd.read_csv('/kaggle/input/fake-news/Fake_train.csv')
dev= pd.read_csv('/kaggle/input/fake-news/Fake_dev.csv')
test=pd.read_csv("/kaggle/input/fake-news/Fake_test_without_labels.csv")
test_label= pd.read_csv('/kaggle/input/fake-news/Fake_test_with_labels.csv')

In [ ]:
test_label

In [ ]:
test_label['label'].value_counts()

In [ ]:
# Combine all datasets into one DataFrame
combined_data = pd.concat([train, dev, test_label], ignore_index=True)

# Ensure the class column exists and is consistent
# Replace 'class_column_name' with the actual name of the column indicating 'original' or 'fake' class.
# Replace 'text_column_name' with the actual name of the column containing text.

class_column = 'label'  # Change to the correct column name
text_column = 'text'    # Change to the correct column name

# Filter rows by class and compute total word count
def count_words_by_class(data, label):
    # Filter rows by the class label
    class_data = data[data[class_column] == label]
    
    # Combine all text for the class into a single string and count words
    total_words = class_data[text_column].str.split().str.len().sum()
    return total_words

# Count words for each class
total_original_words = count_words_by_class(combined_data, 'original')
total_fake_words = count_words_by_class(combined_data, 'Fake')
total_words = total_original_words + total_fake_words
# Print the results
print(f"Total words in 'original' class: {total_original_words}")
print(f"Total words in 'fake' class: {total_fake_words}")
print(f"Total words in the dataset: {total_words}")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Count of duplicate texts
train['text'].duplicated().sum()


In [ ]:
!pip install ai4bharat-transliteration

In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from ai4bharat.transliteration import XlitEngine
from tqdm import tqdm
!pip install malaya

from malaya.text.function import STOPWORDS

stop_words = STOPWORDS
# Ensure necessary resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

e = XlitEngine("ml", beam_width=10)


# Define punctuations to remove
punctuations = '''’'!()-[]{};:'"\,<>./?@#$%^&*_~�'''

# Function to remove URLs and punctuations
def remove_punctuation_url(text):
    text = text.lower()
    # Remove URLs
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    # Remove unwanted characters
    text = text.replace('\n', ' ')
    # Remove punctuations
    text = ''.join(char for char in text if char not in punctuations)
    return text

# Function to remove stopwords
def remove_stopwords(text):
    tokens = word_tokenize(text.lower())
    tokens_without_sw = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens_without_sw)

# Function to remove HTML tags
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

# Function to transliterate text into Malayalam
def transliterate_to_malayalam(text):
    result = e.translit_sentence(text)
    return result["ml"]
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Master function for preprocessing
def full_preprocessing(text):
    text = remove_html_tags(text)  
    text = remove_punctuation_url(text)  # Remove URLs and punctuations
    text = remove_stopwords(text)  # Remove stopwords
    text = transliterate_to_malayalam(text)  # Transliterate to Malayalam
    return text

# Apply preprocessing on training, validation, and test datasets
tqdm.pandas(desc="Processing Text")

train['cleaned'] = train['text'].progress_apply(full_preprocessing)
dev['cleaned'] = dev['text'].progress_apply(full_preprocessing)
test['cleaned'] = test['text'].progress_apply(full_preprocessing)



In [ ]:
from sklearn.preprocessing import LabelEncoder
#original:1 and Fake:0
# Initialize LabelEncoder
encoder = LabelEncoder()

# Fit and transform the labels
train['label'] = encoder.fit_transform(train['label'])
dev['label'] = encoder.transform(dev['label'])

# Verify the conversion
print(train['label'].value_counts())
print(dev['label'].value_counts())


In [ ]:
train

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=2)

# Tokenize datasets
max_length = 60
train_encodings = tokenizer(list(train['text']), truncation=True, padding=True, max_length=max_length)
dev_encodings = tokenizer(list(dev['text']), truncation=True, padding=True, max_length=max_length)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train['label']
)).shuffle(len(train)).batch(16)

dev_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dev_encodings),
    dev['label']
)).batch(16)

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder

model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
def tokenize_texts(texts, tokenizer, max_length=60):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

train_texts = tokenize_texts(train['text'].tolist(), tokenizer)
dev_texts = tokenize_texts(dev['text'].tolist(), tokenizer)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train['label'].tolist())
dev_labels = tf.convert_to_tensor(dev['label'].tolist())


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Prepare data
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_texts["input_ids"], "attention_mask": train_texts["attention_mask"]},
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": dev_texts["input_ids"], "attention_mask": dev_texts["attention_mask"]},
    dev_labels
)).batch(16)

# Custom training loop
epochs = 12
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    
    # Training
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True).logits
            loss = loss_fn(y_batch_train, logits)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch_train, logits)

    train_acc = train_acc_metric.result()
    print(f"Training accuracy: {float(train_acc):.4f}")
    train_acc_metric.reset_state()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Separate labels from the dev dataset
dev_texts = dev['text'].tolist()  # Texts for prediction
dev_labels = dev['label'].tolist()  # True labels

# Tokenize dev texts
dev_inputs = tokenize_texts(dev_texts, tokenizer)

# Predict on dev texts
dev_logits = model.predict({"input_ids": dev_inputs["input_ids"], 
                            "attention_mask": dev_inputs["attention_mask"]}).logits

# Convert logits to predicted class labels
predicted_labels = tf.argmax(dev_logits, axis=1).numpy()

# Evaluate model performance
accuracy = accuracy_score(dev_labels, predicted_labels)
f1 = f1_score(dev_labels, predicted_labels, average="weighted")
report = classification_report(dev_labels, predicted_labels, target_names=["Class 0", "Class 1"])

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Step 1: Ensure the correct labels and predicted labels are available
test['correct_label'] = test_label['label']  # Add the true labels to the test DataFrame

# Step 2: Evaluate the model's performance
accuracy = accuracy_score(test['correct_label'], test['label'])
precision = precision_score(test['correct_label'], test['label'], average='weighted')
recall = recall_score(test['correct_label'], test['label'], average='weighted')
f1 = f1_score(test['correct_label'], test['label'], average='weighted')
report = classification_report(test['correct_label'], test['label'], target_names=["Fake", "original"])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:\n", report)

# Step 3: Create a comparison DataFrame for the first 5 rows
comparison_df = test[['text', 'correct_label', 'label']].head(5).rename(
    columns={'text': 'News', 'correct_label': 'True Label', 'label': 'Predicted Label'}
)

# Step 4: Display the comparison
print("\nComparison of the first 5 rows:")
print(comparison_df)

# Optionally, you can save the comparison DataFrame for reference
comparison_df.to_csv('comparison_first_5.csv', index=False)


In [ ]:
print("\nSample Predictions vs Actual Labels:")
for i in range(50):  
    print(f"Predicted: {predicted_labels[i]}, Actual: {dev_labels[i]}")
    print("-" * 50)

In [ ]:
import os
import zipfile
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Step 1: Get the test texts
test_texts = test['text'].tolist()
test_inputs = tokenize_texts(test_texts, tokenizer)

# Step 2: Predict labels for test data
test_logits = model.predict({
    "input_ids": test_inputs["input_ids"], 
    "attention_mask": test_inputs["attention_mask"]
}).logits

# Convert logits to predicted class labels (numerical)
test['label'] = tf.argmax(test_logits, axis=1).numpy()

# Step 3: Map numerical labels back to "fake" and "original"
label_mapping = {0: "Fake", 1: "original"}
test['label'] = test['label'].map(label_mapping)

# Step 4: Create submission CSV files for multiple runs
team_name = "One_by_zero"
submission_dir = f"{team_name}"
os.makedirs(submission_dir, exist_ok=True)

submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run1.csv"
test[['Id', 'label']].rename(columns={'label': 'Labels'}).to_csv(submission_file, index=False)

# Step 5: Zip the submission files
zip_filename = f"{team_name}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run1.csv"
    zipf.write(submission_file)

print(f"Submission file {zip_filename} created successfully.")


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "eliasedwin7/MalayalamBERT"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the PyTorch model into TensorFlow
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, from_pt=True)

def tokenize_texts(texts, tokenizer, max_length=128):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

train_texts = tokenize_texts(train['text'].tolist(), tokenizer)
dev_texts = tokenize_texts(dev['text'].tolist(), tokenizer)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train['label'].tolist())
dev_labels = tf.convert_to_tensor(dev['label'].tolist())


optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Prepare data
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_texts["input_ids"], "attention_mask": train_texts["attention_mask"]},
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": dev_texts["input_ids"], "attention_mask": dev_texts["attention_mask"]},
    dev_labels
)).batch(16)

# Custom training loop
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    
    # Training
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True).logits
            loss = loss_fn(y_batch_train, logits)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch_train, logits)

    train_acc = train_acc_metric.result()
    print(f"Training accuracy: {float(train_acc):.4f}")
    train_acc_metric.reset_state()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Separate labels from the dev dataset
dev_texts = dev['text'].tolist()  # Texts for prediction
dev_labels = dev['label'].tolist()  # True labels

# Tokenize dev texts
dev_inputs = tokenize_texts(dev_texts, tokenizer)

# Predict on dev texts
dev_logits = model.predict({"input_ids": dev_inputs["input_ids"], 
                            "attention_mask": dev_inputs["attention_mask"]}).logits

# Convert logits to predicted class labels
predicted_labels = tf.argmax(dev_logits, axis=1).numpy()

# Evaluate model performance
accuracy = accuracy_score(dev_labels, predicted_labels)
f1 = f1_score(dev_labels, predicted_labels, average="weighted")
report = classification_report(dev_labels, predicted_labels, target_names=["Class 0", "Class 1"])

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:")
print(report)


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Specify the model name
model_name = "l3cube-pune/malayalam-bert"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model (with PyTorch weights if TensorFlow weights are unavailable)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, from_pt=True)

# Function to tokenize texts
def tokenize_texts(texts, tokenizer, max_length=128):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

# Tokenize training and validation texts
train_texts = tokenize_texts(train['text'].tolist(), tokenizer)
dev_texts = tokenize_texts(dev['text'].tolist(), tokenizer)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train['label'].tolist())
dev_labels = tf.convert_to_tensor(dev['label'].tolist())

# Define optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Prepare data
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_texts["input_ids"], "attention_mask": train_texts["attention_mask"]},
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": dev_texts["input_ids"], "attention_mask": dev_texts["attention_mask"]},
    dev_labels
)).batch(16)

# Custom training loop
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    
    # Training
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True).logits
            loss = loss_fn(y_batch_train, logits)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch_train, logits)

    # Display training accuracy
    train_acc = train_acc_metric.result()
    print(f"Training accuracy: {float(train_acc):.4f}")
    train_acc_metric.reset_state()


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score


# Separate labels from the dev dataset
dev_texts = dev['text'].tolist()  # Texts for prediction
dev_labels = dev['label'].tolist()  # True labels

# Tokenize dev texts
dev_inputs = tokenize_texts(dev_texts, tokenizer)

# Predict on dev texts
dev_logits = model.predict({"input_ids": dev_inputs["input_ids"], 
                            "attention_mask": dev_inputs["attention_mask"]}).logits

# Convert logits to predicted class labels
predicted_labels = tf.argmax(dev_logits, axis=1).numpy()

# Evaluate model performance
accuracy = accuracy_score(dev_labels, predicted_labels)
f1 = f1_score(dev_labels, predicted_labels, average="weighted")
report = classification_report(dev_labels, predicted_labels, target_names=["Class 0", "Class 1"])

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:")
print(report)


In [ ]:
import os
import zipfile
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Step 1: Get the test texts
test_texts = test['text'].tolist()
test_inputs = tokenize_texts(test_texts, tokenizer)

# Step 2: Predict labels for test data
test_logits = model.predict({
    "input_ids": test_inputs["input_ids"], 
    "attention_mask": test_inputs["attention_mask"]
}).logits

# Convert logits to predicted class labels (numerical)
test['label'] = tf.argmax(test_logits, axis=1).numpy()

# Step 3: Map numerical labels back to "fake" and "original"
label_mapping = {0: "Fake", 1: "original"}
test['label'] = test['label'].map(label_mapping)

# Step 4: Create submission CSV files for multiple runs
team_name = "One_by_zero"
submission_dir = f"{team_name}"
os.makedirs(submission_dir, exist_ok=True)

submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run1.csv"
test[['Id', 'label']].rename(columns={'label': 'Labels'}).to_csv(submission_file, index=False)

# Step 5: Zip the submission files
zip_filename = f"{team_name}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run1.csv"
    zipf.write(submission_file)

print(f"Submission file {zip_filename} created successfully.")


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the test labels (ground truth)
test_labels = test_label['label'].tolist()  # The true labels

# Get the predicted labels from your model (from 'test')
predicted_labels = test['label'].tolist()  # The predicted labels

# Generate the confusion matrix
cm = confusion_matrix(test_labels, predicted_labels, labels=["Fake", "original"])

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Fake", "original"], yticklabels=["Fake", "original"])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

# Optionally, print the confusion matrix as well
print("Confusion Matrix:")
print(cm)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Step 1: Ensure the correct labels and predicted labels are available
test['correct_label'] = test_label['label']  # Add the true labels to the test DataFrame

# Step 2: Evaluate the model's performance
accuracy = accuracy_score(test['correct_label'], test['label'])
precision = precision_score(test['correct_label'], test['label'], average='weighted')
recall = recall_score(test['correct_label'], test['label'], average='weighted')
f1 = f1_score(test['correct_label'], test['label'], average='weighted')
report = classification_report(test['correct_label'], test['label'], target_names=["Fake", "original"])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:\n", report)

# Step 3: Create a comparison DataFrame for the first 5 rows
comparison_df = test[['text', 'correct_label', 'label']].head(5).rename(
    columns={'text': 'News', 'correct_label': 'True Label', 'label': 'Predicted Label'}
)

# Step 4: Display the comparison
print("\nComparison of the first 5 rows:")
print(comparison_df)

# Optionally, you can save the comparison DataFrame for reference
comparison_df.to_csv('comparison_first_5.csv', index=False)


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Load the tokenizer and model for IndicBERT
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-only")

model = TFAutoModelForSequenceClassification.from_pretrained(
    "ai4bharat/IndicBERTv2-MLM-only", 
    num_labels=2,from_pt=True  # Adjust the number of labels as per your classification task
)

# Function to tokenize texts
def tokenize_texts(texts, tokenizer, max_length=60):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

# Tokenize training and validation texts
train_texts = tokenize_texts(train['text'].tolist(), tokenizer)
dev_texts = tokenize_texts(dev['text'].tolist(), tokenizer)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train['label'].tolist())
dev_labels = tf.convert_to_tensor(dev['label'].tolist())

# Define optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Prepare data
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_texts["input_ids"], "attention_mask": train_texts["attention_mask"]},
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": dev_texts["input_ids"], "attention_mask": dev_texts["attention_mask"]},
    dev_labels
)).batch(16)

# Custom training loop
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    
    # Training
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True).logits
            loss = loss_fn(y_batch_train, logits)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch_train, logits)

    # Display training accuracy
    train_acc = train_acc_metric.result()
    print(f"Training accuracy: {float(train_acc):.4f}")
    train_acc_metric.reset_state()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Separate labels from the dev dataset
dev_texts = dev['text'].tolist()  # Texts for prediction
dev_labels = dev['label'].tolist()  # True labels

# Tokenize dev texts
dev_inputs = tokenize_texts(dev_texts, tokenizer)

# Predict on dev texts
dev_logits = model.predict({"input_ids": dev_inputs["input_ids"], 
                            "attention_mask": dev_inputs["attention_mask"]}).logits

# Convert logits to predicted class labels
predicted_labels = tf.argmax(dev_logits, axis=1).numpy()

# Evaluate model performance
accuracy = accuracy_score(dev_labels, predicted_labels)
f1 = f1_score(dev_labels, predicted_labels, average="weighted")
report = classification_report(dev_labels, predicted_labels, target_names=["Class 0", "Class 1"])

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Step 1: Ensure the correct labels and predicted labels are available
test['correct_label'] = test_label['label']  # Add the true labels to the test DataFrame

# Step 2: Evaluate the model's performance
accuracy = accuracy_score(test['correct_label'], test['label'])
precision = precision_score(test['correct_label'], test['label'], average='weighted')
recall = recall_score(test['correct_label'], test['label'], average='weighted')
f1 = f1_score(test['correct_label'], test['label'], average='weighted')
report = classification_report(test['correct_label'], test['label'], target_names=["Fake", "original"])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:\n", report)

# Step 3: Create a comparison DataFrame for the first 5 rows
comparison_df = test[['text', 'correct_label', 'label']].head(5).rename(
    columns={'text': 'News', 'correct_label': 'True Label', 'label': 'Predicted Label'}
)

# Step 4: Display the comparison
print("\nComparison of the first 5 rows:")
print(comparison_df)

# Optionally, you can save the comparison DataFrame for reference
comparison_df.to_csv('comparison_first_5.csv', index=False)


In [ ]:
import os
import zipfile
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Step 1: Get the test texts
test_texts = test['text'].tolist()
test_inputs = tokenize_texts(test_texts, tokenizer)

# Step 2: Predict labels for test data
test_logits = model.predict({
    "input_ids": test_inputs["input_ids"], 
    "attention_mask": test_inputs["attention_mask"]
}).logits

# Convert logits to predicted class labels (numerical)
test['label'] = tf.argmax(test_logits, axis=1).numpy()

# Step 3: Map numerical labels back to "fake" and "original"
label_mapping = {0: "Fake", 1: "original"}
test['label'] = test['label'].map(label_mapping)

# Step 4: Create submission CSV files for multiple runs
team_name = "One_by_zero"
submission_dir = f"{team_name}"
os.makedirs(submission_dir, exist_ok=True)

submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run2.csv"
test[['Id', 'label']].rename(columns={'label': 'Labels'}).to_csv(submission_file, index=False)

# Step 5: Zip the submission files
zip_filename = f"{team_name}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run2.csv"
    zipf.write(submission_file)

print(f"Submission file {zip_filename} created successfully.")


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Load the tokenizer and model for FakeNews-Classifier-GPT-Labels
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

model = TFAutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", 
    num_labels=2,  # Adjust if your task involves more than two labels
    from_pt=True
)


  



In [ ]:
def tokenize_texts(texts, tokenizer, max_length=60):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

train_texts = tokenize_texts(train['text'].tolist(), tokenizer)
dev_texts = tokenize_texts(dev['text'].tolist(), tokenizer)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train['label'].tolist())
dev_labels = tf.convert_to_tensor(dev['label'].tolist())


optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Prepare data
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_texts["input_ids"], "attention_mask": train_texts["attention_mask"]},
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": dev_texts["input_ids"], "attention_mask": dev_texts["attention_mask"]},
    dev_labels
)).batch(16)

# Custom training loop
epochs = 15
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    
    # Training
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True).logits
            loss = loss_fn(y_batch_train, logits)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch_train, logits)

    train_acc = train_acc_metric.result()
    print(f"Training accuracy: {float(train_acc):.4f}")
    train_acc_metric.reset_state()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Separate labels from the dev dataset
dev_texts = dev['text'].tolist()  # Texts for prediction
dev_labels = dev['label'].tolist()  # True labels

# Tokenize dev texts
dev_inputs = tokenize_texts(dev_texts, tokenizer)

# Predict on dev texts
dev_logits = model.predict({"input_ids": dev_inputs["input_ids"], 
                            "attention_mask": dev_inputs["attention_mask"]}).logits

# Convert logits to predicted class labels
predicted_labels = tf.argmax(dev_logits, axis=1).numpy()

# Evaluate model performance
accuracy = accuracy_score(dev_labels, predicted_labels)
f1 = f1_score(dev_labels, predicted_labels, average="weighted")
report = classification_report(dev_labels, predicted_labels, target_names=["Class 0", "Class 1"])

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:")
print(report)


In [ ]:
import os
import zipfile
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Step 1: Get the test texts
test_texts = test['text'].tolist()
test_inputs = tokenize_texts(test_texts, tokenizer)

# Step 2: Predict labels for test data
test_logits = model.predict({
    "input_ids": test_inputs["input_ids"], 
    "attention_mask": test_inputs["attention_mask"]
}).logits

# Convert logits to predicted class labels (numerical)
test['label'] = tf.argmax(test_logits, axis=1).numpy()

# Step 3: Map numerical labels back to "fake" and "original"
label_mapping = {0: "Fake", 1: "original"}
test['label'] = test['label'].map(label_mapping)

# Step 4: Create submission CSV files for multiple runs
team_name = "One_by_zero"
submission_dir = f"{team_name}"
os.makedirs(submission_dir, exist_ok=True)

submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run2.csv"
test[['Id', 'label']].rename(columns={'label': 'Labels'}).to_csv(submission_file, index=False)

# Step 5: Zip the submission files
zip_filename = f"{team_name}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run2.csv"
    zipf.write(submission_file)

print(f"Submission file {zip_filename} created successfully.")


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Step 1: Ensure the correct labels and predicted labels are available
test['correct_label'] = test_label['label']  # Add the true labels to the test DataFrame

# Step 2: Evaluate the model's performance
accuracy = accuracy_score(test['correct_label'], test['label'])
precision = precision_score(test['correct_label'], test['label'], average='weighted')
recall = recall_score(test['correct_label'], test['label'], average='weighted')
f1 = f1_score(test['correct_label'], test['label'], average='weighted')
report = classification_report(test['correct_label'], test['label'], target_names=["Fake", "original"])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:\n", report)

# Step 3: Create a comparison DataFrame for the first 5 rows
comparison_df = test[['text', 'correct_label', 'label']].head(5).rename(
    columns={'text': 'News', 'correct_label': 'True Label', 'label': 'Predicted Label'}
)

# Step 4: Display the comparison
print("\nComparison of the first 5 rows:")
print(comparison_df)

# Optionally, you can save the comparison DataFrame for reference
comparison_df.to_csv('comparison_first_5.csv', index=False)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/malayalam-sentence-bert-nli")

model = AutoModelForSequenceClassification.from_pretrained(
    "l3cube-pune/malayalam-sentence-bert-nli",
    num_labels=2  # Adjust based on your task
)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch # Ensure you use AdamW from transformers
from torch.nn import CrossEntropyLoss
from transformers import AdamW
def tokenize_texts(texts, tokenizer, max_length=60):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"  # Change to PyTorch tensors
    )

train_texts = tokenize_texts(train['text'].tolist(), tokenizer)
dev_texts = tokenize_texts(dev['text'].tolist(), tokenizer)

# Convert labels to tensors (assuming they are already numeric)
train_labels = torch.tensor(train['label'].tolist())
dev_labels = torch.tensor(dev['label'].tolist())

# Create DataLoader for batching
train_dataset = TensorDataset(train_texts["input_ids"], train_texts["attention_mask"], train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

dev_dataset = TensorDataset(dev_texts["input_ids"], dev_texts["attention_mask"], dev_labels)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

# Define optimizer, loss function, and device
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Custom training loop
epochs = 15
for epoch in range(epochs):
    model.train()  # Set model to training mode
    total_train_loss = 0
    total_train_correct = 0
    total_train_samples = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Compute loss and backpropagate
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        # Track training accuracy
        preds = torch.argmax(logits, dim=1)
        total_train_correct += (preds == labels).sum().item()
        total_train_samples += labels.size(0)
        total_train_loss += loss.item()

    train_accuracy = total_train_correct / total_train_samples
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {total_train_loss / len(train_dataloader):.4f}, Train Accuracy: {train_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Set model to evaluation mode
model.eval()

# Separate texts and labels from the dev dataset
dev_texts_list = dev['text'].tolist()  # Texts for prediction
dev_labels_list = dev['label'].tolist()  # True labels

# Tokenize dev texts
dev_inputs = tokenize_texts(dev_texts_list, tokenizer)

# Move inputs to the same device as the model
input_ids = dev_inputs["input_ids"].to(device)
attention_mask = dev_inputs["attention_mask"].to(device)

# Perform inference without gradient computation
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Convert logits to predicted class labels
predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

# Evaluate model performance
accuracy = accuracy_score(dev_labels_list, predicted_labels)
f1 = f1_score(dev_labels_list, predicted_labels, average="weighted")
report = classification_report(dev_labels_list, predicted_labels, target_names=["Class 0", "Class 1"])

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Classification Report:")
print(report)


In [ ]:
import os
import zipfile
from torch.utils.data import DataLoader

# Step 1: Ensure model is in evaluation mode
model.eval()

# Step 2: Predict labels for test data
test_texts = test['text'].tolist()
test_inputs = tokenize_texts(test_texts, tokenizer)

# Move test inputs to the same device as the model
input_ids = test_inputs["input_ids"].to(device)
attention_mask = test_inputs["attention_mask"].to(device)

# Perform inference without gradient computation
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    test_logits = outputs.logits

# Convert logits to predicted class labels
predicted_labels = torch.argmax(test_logits, dim=1).cpu().numpy()

# Map numerical labels back to string labels
label_mapping = {0: "Fake", 1: "original"}
test['label'] = [label_mapping[label] for label in predicted_labels]

# Step 3: Create submission CSV file
team_name = "One_by_zero"
submission_dir = f"{team_name}"
os.makedirs(submission_dir, exist_ok=True)

submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run3.csv"
test[['Id', 'label']].rename(columns={'label': 'Labels'}).to_csv(submission_file, index=False)

# Step 4: Zip the submission files
zip_filename = f"{team_name}.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    submission_file = f"{submission_dir}/{team_name}_Malayalam_Task1_run3.csv"
    zipf.write(submission_file)

print(f"Submission file {zip_filename} created successfully.")


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Step 1: Ensure the correct labels and predicted labels are available
test['correct_label'] = test_label['label']  # Add the true labels to the test DataFrame

# Step 2: Evaluate the model's performance
accuracy = accuracy_score(test['correct_label'], test['label'])
precision = precision_score(test['correct_label'], test['label'], average='weighted')
recall = recall_score(test['correct_label'], test['label'], average='weighted')
f1 = f1_score(test['correct_label'], test['label'], average='weighted')
report = classification_report(test['correct_label'], test['label'], target_names=["Fake", "original"])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:\n", report)

# Step 3: Create a comparison DataFrame for the first 5 rows
comparison_df = test[['text', 'correct_label', 'label']].head(5).rename(
    columns={'text': 'News', 'correct_label': 'True Label', 'label': 'Predicted Label'}
)

# Step 4: Display the comparison
print("\nComparison of the first 5 rows:")
print(comparison_df)

# Optionally, you can save the comparison DataFrame for reference
comparison_df.to_csv('comparison_first_5.csv', index=False)
